**INSTALLING MODULES**

In [ ]:
# Install and authenticate Google Earth Engine
!pip install earthengine-api
!earthengine authenticate

In [ ]:
!pip install geemap
!pip install rasterio
!pip install folium

In [ ]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

**LOADING MODULES**

In [ ]:
# Import necessary libraries
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geemap
from PIL import Image, ImageDraw, ImageFont
import imageio
import os
import rasterio
import folium
from folium import raster_layers
from selenium import webdriver


In [ ]:
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

In [ ]:
# Define the region of interest: Sahel
sahel_region = ee.Geometry.Polygon(
        [[[-12, 15],
          [-12, 21],
          [15, 21],
          [15, 15]]])

# Define the time range
start_year = 2000
end_year = 2023

In [ ]:
# Function to get MODIS NDVI data for a specific year
def get_modis_ndvi(year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    # Use MOD13A2.006 product for NDVI
    dataset = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(start_date, end_date).filterBounds(sahel_region)

    # Extract NDVI and scale it. MODIS NDVI values are scaled by a factor of 0.0001
    ndvi = dataset.select('NDVI').mean().multiply(0.0001)

    return ndvi

In [ ]:
# Extract yearly NDVI values
yearly_ndvi = {}
for year in range(start_year, end_year+1):
    ndvi_image = get_modis_ndvi(year)
    ndvi_value = ndvi_image.reduceRegion(reducer=ee.Reducer.mean(),
                                         geometry=sahel_region, scale=1000,
                                         maxPixels=1e13).get('NDVI').getInfo()
    yearly_ndvi[year] = ndvi_value

In [ ]:
# Convert to DataFrame for analysis
df = pd.DataFrame(list(yearly_ndvi.items()), columns=['Year', 'NDVI'])

# Export to CSV
df.to_csv('sahel_modis_ndvi_2000_2023.csv', index=False)


**EXPORTING IMAGES, ONE FOR EACH YEAR**

In [ ]:


# Initialize the Earth Engine module
ee.Initialize()

# Function to get MODIS NDVI data for a specific year
def get_modis_ndvi(year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    # Use MOD13A2.006 product for NDVI
    dataset = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(start_date, end_date).filterBounds(sahel_region)

    # Extract NDVI and scale it. MODIS NDVI values are scaled by a factor of 0.0001
    ndvi = dataset.select('NDVI').mean().multiply(0.0001)

    return ndvi

# Export each yearly average NDVI image
image_list = []
for year in range(2000, 2024):
    ndvi_image = get_modis_ndvi(year)
    out_path = f'/content/NDVI_{year}.tif'
    geemap.ee_export_image(ndvi_image, filename=out_path, scale=700, region=sahel_region, file_per_band=False)
    image_list.append(out_path)




**VISUALIZE SOME OF THE IMAGES USING FOLIUM**

In [ ]:


# Function to overlay a GeoTIFF on a folium map
def add_geotiff_to_map(m, img_path, colormap, vmin=-0.1, vmax=0.9, opacity=0.6):
    with rasterio.open(img_path) as src:
        # Get bounds of the image
        bounds = src.bounds
        # Read the image data
        data = src.read(1)
        # Normalize the data to fit within the range [0, 1]
        norm_data = (data - vmin) / (vmax - vmin)
        # Apply the colormap to the normalized data to get an RGB image
        colored_data = (colormap(norm_data)[:, :, :3] * 255).astype(np.uint8)
        # Use folium's rasters method to overlay the image on the map
        raster_layers.ImageOverlay(
            image=colored_data,
            bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
            opacity=opacity
        ).add_to(m)

# Define a colormap that transitions from yellow to red
colormap = plt.cm.YlOrRd

# Create a list to store the maps
maps = []

# Overlay the GeoTIFFs on individual maps
for year in [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
             2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
             2022, 2023]:
    m = folium.Map(location=[18, 1.5], zoom_start=5, tiles='Stamen Terrain')  # Added Stamen Terrain tiles here
    img_path = f'/content/NDVI_{year}.tif'
    add_geotiff_to_map(m, img_path, colormap)
    maps.append(m)

# Display the maps
for m in maps:
    display(m)


**CROPPING THE IMAGES**

In [ ]:




# Function to overlay a GeoTIFF on a folium map
def add_geotiff_to_map(m, img_path, colormap, vmin=-0.1, vmax=0.9, opacity=0.6):
    with rasterio.open(img_path) as src:
        bounds = src.bounds
        data = src.read(1)
        norm_data = (data - vmin) / (vmax - vmin)
        colored_data = (colormap(norm_data)[:, :, :3] * 255).astype(np.uint8)
        raster_layers.ImageOverlay(
            image=colored_data,
            bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
            opacity=opacity
        ).add_to(m)

colormap = plt.cm.YlOrRd

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

def save_folium_map_as_image(m, filename, year):
    tmp_path = "temp_map.html"
    m.save(tmp_path)
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(f"file://{os.getcwd()}/{tmp_path}")
    browser.save_screenshot(filename)
    browser.quit()

    # Crop the saved image and add the year label
    with Image.open(filename) as img:
        # Define crop coordinates (left, upper, right, lower)
        # Adjust these values as needed
        cropped_img = img.crop((100, 100, 700, 500))

        # Add the year label to the cropped image
        draw = ImageDraw.Draw(cropped_img)
        font = ImageFont.load_default()
        draw.text((20, 20), str(year), font=font, fill="black")

        cropped_img.save(filename)

maps = []

for year in range(2000, 2024):
    m = folium.Map(location=[18, 1.5], zoom_start=5, tiles='Stamen Terrain')
    img_path = f'/content/NDVI_{year}.tif'
    add_geotiff_to_map(m, img_path, colormap)
    maps.append(m)
    save_folium_map_as_image(m, f'NDVI_{year}.png', year)




**CREATE THE GIF**

In [ ]:


# Directory where the images are saved
image_dir = '/content/'

# List to store the images
images = []


# Loop through each year and read the images
for year in range(2000, 2024):
    image_path = os.path.join(image_dir, f'NDVI_{year}.png')

    # Open the image using PIL
    img = Image.open(image_path)

    # Add the year label to the image
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()
    draw.text((20, 20), str(year), font=font, fill="black")

    # Append to the images list
    images.append(img)

# Create the GIF
out_gif_path = os.path.join(image_dir, 'NDVI_timelapse.gif')
images[0].save(out_gif_path, save_all=True, append_images=images[1:], loop=0, duration=1000)
